# Technique for Order of Preference by Similarity to Ideal Solution (AHP-TOPSIS)

##### Modified from: https://www.kaggle.com/code/hungrybluedev/topsis-implementation/notebook </br>

It is a multi-criteria decision analysis method that is based on the concept that the chosen alternative should have the shortest geometric distance to the Positive Ideal Solution (PIS) and the longest geometric solution from the Negative Ideal Solution (NIS).

# AHP Weight calculation
1.  Pair-wise comparison of each criteria and sub-criteria to establish the weight of the supply chain parameters.
2. Global summation of all these weights (weighted arithmetic sum) for each alternative and ordering them on the basis of this weighted sum.
3. Calculate the consistency ratio which should be less than 0.10, otherwise the weights are not balanced

## AHP Pair-wise Comparison Matrix
As the first step a pairwise comparison matrix is determined qualitatively according to the priority using the Saaty 9 point scale (Evelyn, E. et.al, 2015). This table is subjectively evaluated, as AHP combines both quantitative and qualitative aspects in the decision method, which helps the analysis to find the best possible answer rather than a correct solution (Longaray, A.A. et.al, 2015).
<br /><br />

| Features              | Digital Prepardness  | Natural Disaster | Labor Strikes | Political Instability | Logistic Index |
|-----------------------|----------------------|------------------|---------------|-----------------------|----------------|
| Digital Prepardness   | 1                    | 6                | 8             | 4                     | 3              |
| Natural Disaster      | 1/6                  | 1                | 3             | 1/2                   | 1/4            |
| Labor Strikes         | 1/8                  | 1/3              | 1             | 1/2                   | 1/4            |
| Political Instability | 1/4                  | 2                | 2             | 1                     | 1/3            |
| Logistic Index        | 1/3                  | 5                | 4             | 3                     | 1              |

In [30]:
from ahpy import ahpy

supply_chain_comp = {
    ('Digital Prepardness', 'Digital Prepardness'): 1, ('Digital Prepardness', 'Natural Disasters'): 6, ('Digital Prepardness', 'Labor Strikes'): 8, ('Digital Prepardness', 'Political Instability'): 4, ('Digital Prepardness', 'Logistic Index'): 3,
    ('Natural Disasters', 'Digital Prepardness'): 1/6, ('Natural Disasters', 'Natural Disasters'): 1, ('Natural Disasters', 'Labor Strikes'): 3, ('Natural Disasters', 'Political Instability'): 1/2, ('Natural Disasters', 'Logistic Index'): 1/5,
    ('Labor Strikes', 'Digital Prepardness'): 1/8, ('Labor Strikes', 'Natural Disasters'): 1/3, ('Labor Strikes', 'Labor Strikes'): 1, ('Labor Strikes', 'Political Instability'): 1/2, ('Labor Strikes', 'Logistic Index'): 1/4,
    ('Political Instability', 'Digital Prepardness'): 1/4, ('Political Instability', 'Natural Disasters'): 2, ('Political Instability', 'Labor Strikes'): 2, ('Political Instability', 'Political Instability'): 1, ('Political Instability', 'Logistic Index'): 1/3,
    ('Logistic Index', 'Digital Prepardness'): 1/3, ('Logistic Index', 'Natural Disasters'): 5, ('Logistic Index', 'Labor Strikes'): 4, ('Logistic Index', 'Political Instability'): 3, ('Logistic Index', 'Logistic Index'): 1
    }


## Calculate the weight for the criteria
The above table represents an n x n comparison matrix which contains the intensities defined by us (Karmaker, C.L et.al, 2018). $$
 M = (w_{i}/w_{j})_{n x n} = (\begin{array}{cc}
w_{1}/w_{1} & w_{1}/w_{2} & .... & w_{1}/w_{n}\\
w_{2}/w_{1} & w_{2}/w_{2} & .... & w_{2}/w_{n}\\
. & . & ..... & . \\
w_{n}/w_{1} & w_{n}/w_{2} & .... & w_{n}/w_{n}
\end{array})
$$

## Generate the weights
The n x n matrix is first normalized using the formula below and a priority vector is generated with which the weights are generated.
$$
W = \begin{bmatrix}
           w_{1} \\
           w_{2} \\
           \vdots \\
           w_{n}
         \end{bmatrix}
$$

In [31]:
supply_chain = ahpy.Compare(name='Supply_Chain', comparisons=supply_chain_comp, precision=3, random_index='saaty')

print(supply_chain.target_weights)
print('Consistency Ratio: ' + str(supply_chain.consistency_ratio))

{'Digital Prepardness': 0.498, 'Logistic Index': 0.26, 'Political Instability': 0.111, 'Natural Disasters': 0.08, 'Labor Strikes': 0.05}
Consistency Ratio: 0.051


## Calculate and check the Consistency ratio
It is assumed that the experts using the AHP makers are objective. However, this is not true in real life which generates a certain level of uncertainty towards this evaluation. Therefore, Saaty solved this issue by creating the consistency Index and consistency ratios. To accept the matrix, the CR value should be less the 0.1 meaning there could be an inconsistency error of 10%.

In [32]:
# All the packages that we need to import
import numpy as np               # for linear algebra
import pandas as pd              # for tabular output
from scipy.stats import rankdata # for ranking the candidates

## Pre-requisites

For this problem, we are always provided with the following data:
1. The ratings in every category for each candidate.
2. The weights for every category or attribute to be considered.

Note that an attribute can be beneficial attribute (in which case, we will want to maximize it's contribution) or a cost attribute (which we will need to minimize). We call the set of beneficial attributes $J_1$ and that of cost attributes $J_2 = J_1^C$.

In [33]:
# The given data encoded into vectors and matrices

attributes = np.array(["Digital Prepardness", "Natural Disasters", "Labor Strikes", "Political Instability", "Logistic Index"])
candidates = np.array(["Germany", "Poland", "France", "Belgium", "USA", "UK", "Portugal", "Bulgaria", "Netherlands", "Spain", "Ireland", "Hungary"])
raw_data = np.array([
    [88.07, 3.26, 17.30, 0.76, 4.10], # Germany
    [84.09, 3.65, 16.0, 0.51, 3.6], # Poland
    [86.41, 3.53, 127.6, 0.37, 3.9], # France
    [90.69, 3.52, 97.9, 0.61, 4.], # Belgium
    [83.48, 7.6, 17.11, 0.54, 3.7], # USA
    [85.59, 4.058, 17.9, 0.54, 3.8], # UK
    [85.5, 3.9, 14.1, 0., 3.4], # Portugal
    [64.37, 3.88, 2.1, 0.46, 3.2], # Bulgaria
    [84.66, 7.44, 19.2, 0.92, 4.1], # Netherlands
    [88.61, 2.94, 49.1, 0.58, 3.9], # Spain
    [78.56, 4.2, 15.6, 0.86, 3.6], # Ireland
    [76.74, 4.68, 6.2, 0.86, 3.2], # Hungary
])

weights = np.array([0.498, 0.08, 0.05, 0.111, 0.26])

# The indices of the attributes (zero-based) that are considered beneficial.
# Cost benefit functions 0 when cost (lower is better) and 1 when benefit function (more is better)
benefit_attributes = set([0, 1, 2, 3, 4])

# Display the raw data we have
pd.DataFrame(data=raw_data, index=candidates, columns=attributes)

,Digital Prepardness,Natural Disasters,Labor Strikes,Political Instability,Logistic Index
Germany,88.07,3.260,17.30,0.76,4.1
Poland,84.09,3.650,16.00,0.51,3.6
France,86.41,3.530,127.60,0.37,3.9
Belgium,90.69,3.520,97.90,0.61,4.0
USA,83.48,7.600,17.11,0.54,3.7
UK,85.59,4.058,17.90,0.54,3.8
Portugal,85.50,3.900,14.10,0.00,3.4
Bulgaria,64.37,3.880,2.10,0.46,3.2
Netherlands,84.66,7.440,19.20,0.92,4.1
Spain,88.61,2.940,49.10,0.58,3.9


## Step 1 - Normalizing the ratings

$$r_{ij}=\frac{x_{ij}}{\sqrt{\sum_{i = 1}^{m} x_{ij}^2}}$$

where $i = 1, 2, \ldots, m$ and $j = 1, 2, \ldots, n$.

In [34]:
m = len(raw_data)
n = len(attributes)
divisors = np.empty(n)
for j in range(n):
    column = raw_data[:,j]
    divisors[j] = np.sqrt(column @ column)

raw_data /= divisors

columns = ["$X_{%d}$" % j for j in range(n)]
pd.DataFrame(data=raw_data, index=candidates, columns=columns)

,$X_{0}$,$X_{1}$,$X_{2}$,$X_{3}$,$X_{4}$
Germany,0.305059,0.203419,0.099387,0.346765,0.318097
Poland,0.291273,0.227755,0.091919,0.232697,0.279305
France,0.299309,0.220267,0.733054,0.168820,0.302580
Belgium,0.314134,0.219643,0.562429,0.278324,0.310339
USA,0.289160,0.474229,0.098296,0.246385,0.287063
UK,0.296468,0.253213,0.102834,0.246385,0.294822
Portugal,0.296157,0.243355,0.081004,0.000000,0.263788
Bulgaria,0.222966,0.242107,0.012064,0.209884,0.248271
Netherlands,0.293247,0.464246,0.110303,0.419768,0.318097
Spain,0.306929,0.183452,0.282076,0.264636,0.302580


## Step 2 - Calculating the Weighted Normalized Ratings

$$v_{ij} = w_j r_{ij}$$

where $i = 1, 2, \ldots, m$ and $j = 1, 2, \ldots, n$.

In [35]:
raw_data *= weights
pd.DataFrame(data=raw_data, index=candidates, columns=columns)

,$X_{0}$,$X_{1}$,$X_{2}$,$X_{3}$,$X_{4}$
Germany,0.151919,0.016274,0.004969,0.038491,0.082705
Poland,0.145054,0.018220,0.004596,0.025829,0.072619
France,0.149056,0.017621,0.036653,0.018739,0.078671
Belgium,0.156439,0.017571,0.028121,0.030894,0.080688
USA,0.144002,0.037938,0.004915,0.027349,0.074636
UK,0.147641,0.020257,0.005142,0.027349,0.076654
Portugal,0.147486,0.019468,0.004050,0.000000,0.068585
Bulgaria,0.111037,0.019369,0.000603,0.023297,0.064550
Netherlands,0.146037,0.037140,0.005515,0.046594,0.082705
Spain,0.152851,0.014676,0.014104,0.029375,0.078671


## Step 3 - Identifying PIS ($A^*$) and NIS ($A^-$)

$$
\begin{align}
A^* &= \left\{v_1^*, v_2^*, \ldots, v_n^*\right\} \\
A^- &= \left\{v_1^-, v_2^-, \ldots, v_n^-\right\} \\
\end{align}
$$

And we define

$$
\begin{align}
v_j^* &=
\begin{cases}
\max{(v_{ij})}, \text{ if} j \in J_1 \\
\min{(v_{ij})}, \text{ if} j \in J_2
\end{cases}
\\
v_j^- &=
\begin{cases}
\min{(v_{ij})}, \text{ if} j \in J_1 \\
\max{(v_{ij})}, \text{ if} j \in J_2
\end{cases}
\\
\end{align}
$$

where $i = 1, 2, \ldots, m$ and $j = 1, 2, \ldots, n$.

In [36]:
a_pos = np.zeros(n)
a_neg = np.zeros(n)
for j in range(n):
    column = raw_data[:,j]
    max_val = np.max(column)
    min_val = np.min(column)
    
    # See if we want to maximize benefit or minimize cost (for PIS)
    if j in benefit_attributes:
        a_pos[j] = max_val
        a_neg[j] = min_val
    else:
        a_pos[j] = min_val
        a_neg[j] = max_val

pd.DataFrame(data=[a_pos, a_neg], index=["$A^*$", "$A^-$"], columns=columns)

,$X_{0}$,$X_{1}$,$X_{2}$,$X_{3}$,$X_{4}$
$A^*$,0.156439,0.037938,0.036653,0.046594,0.082705
$A^-$,0.111037,0.014676,0.000603,0.000000,0.064550


## Step 4 and 5 - Calculating Separation Measures and Similarities to PIS

The separation or distance between the alternatives can be measured by the $n$-dimensional Euclidean distance. The separation from the PIS $A^*$ and NIS $A^-$ are $S^*$ and $S^-$ respectively.

$$
\begin{align}
S_i^* &= \sqrt{\sum_{j = 1}^n \left(v_{ij} - v^*_j\right)^2} \\
S_i^- &= \sqrt{\sum_{j = 1}^n \left(v_{ij} - v^-_j\right)^2} \\
\end{align}
$$

where $i = 1, 2, \ldots, m$ and $j = 1, 2, \ldots, n$.

We also calculate

$$
C^*_i = \frac{S_i^-}{S_i^* + S_i^-},\text{ where }i = 1, 2, \ldots, m
$$

In [37]:
sp = np.zeros(m)
sn = np.zeros(m)
cs = np.zeros(m)

for i in range(m):
    diff_pos = raw_data[i] - a_pos
    diff_neg = raw_data[i] - a_neg
    sp[i] = np.sqrt(diff_pos @ diff_pos)
    sn[i] = np.sqrt(diff_neg @ diff_neg)
    cs[i] = sn[i] / (sp[i] + sn[i])

pd.DataFrame(data=zip(sp, sn, cs), index=candidates, columns=["$S^*$", "$S^-$", "$C^*$"])

,$S^*$,$S^-$,$C^*$
Germany,0.039488,0.059195,0.599853
Poland,0.045596,0.043794,0.489922
France,0.035489,0.057482,0.618278
Belgium,0.027169,0.063575,0.700597
USA,0.039968,0.049961,0.555556
UK,0.042308,0.047813,0.530540
Portugal,0.062086,0.037144,0.374321
Bulgaria,0.067661,0.023765,0.259936
Netherlands,0.032839,0.065225,0.665130
Spain,0.037084,0.054707,0.595995


## Step 6 - Ranking the candidates/alternatives

We choose the candidate with the maximum $C^*$ or rank all the alternatives in descending order according to their $C^*$ values. This process can also be done for the $S^*$ and $S^-$ values.

In [38]:
def rank_according_to(data):
    ranks = rankdata(data).astype(int)
    ranks -= 1
    return candidates[ranks][::-1]

In [39]:
cs_order = rank_according_to(cs)
sp_order = rank_according_to(sp)
sn_order = rank_according_to(sn)

pd.DataFrame(data=zip(cs_order, sp_order, sn_order), index=range(1, m + 1), columns=["$C^*$", "$S^*$", "$S^-$"])

,$C^*$,$S^*$,$S^-$
1,Belgium,Spain,USA
2,UK,Bulgaria,Portugal
3,Bulgaria,Belgium,Bulgaria
4,Ireland,Poland,Hungary
5,Germany,Hungary,Germany
6,Poland,Ireland,Poland
7,USA,Portugal,Belgium
8,Portugal,UK,UK
9,Hungary,Germany,Ireland
10,Spain,France,Netherlands


In [40]:
print("The best candidate/alternative according to C* is " + cs_order[0])
print("The preferences in descending order are " + ", ".join(cs_order) + ".")

The best candidate/alternative according to C* is Belgium
The preferences in descending order are Belgium, UK, Bulgaria, Ireland, Germany, Poland, USA, Portugal, Hungary, Spain, France, Netherlands.


## References
Evelyn, E. and EdmondYeboah, N., (2015). Ranking Agricultural Supply Chain Risk in Ghana: An AHP Approach. International Journal of Economics,Commerece and Management, III, 2, pp.1-12.

Hungrybluedev (2021) Topsis implementation, Kaggle. Available from: https://www.kaggle.com/code/hungrybluedev/topsis-implementation/notebook (Accessed: 12 May 2023).

Karmaker, C.L., Ahmed, S.M.T., Rahman, M.S., Tahiduzzaman, M., Biswas, T.K., Rahman, M. and Biswas, S.K., 2018. A framework of faculty performance evaluation: A case study in Bangladesh. International Journal of Research in Advanced Engineering and Technology, 4(3), pp.18-24.

Longaray, A.A., Gois, J.D.D.R. and da Silva Munhoz, P.R., ()2015. Proposal for using AHP method to evaluate the quality of services provided by outsourced companies. Procedia Computer Science, 55, pp.715-724.